## Definições
Bibliotecas utilizadas e definições de path. Para facilitar a manipulação de arquivos, o diretório do projeto é definido neste ponto. Também é realizada a carga dos dados brutos, que serão processados para gerar o conjunto de dados interpretável que alimentará a modelagem.

In [3]:
import numpy as np
import pandas as pd
#base_path = "C:/Users/99818854/Projetos/GitRep/adaptive_learning"
base_path = "/media/bruno/Arquivos/Desenvolvimento/NextQuestion"

In [4]:
df = pd.read_csv(f"{base_path}/data/Dataset_model.csv", low_memory=False)

## Estratégia
Carregar todos os dados disponíveis na base para uma modelagem não é particularmente interessante, principalmente pela quantidade de dimensões e a falta de valor de negócio em alguns campos. Pensando em aprendizagem adaptativa, podemos definir esse termo como o monitoramento da maestria adquirida por uma pessoa ao interagir com conteúdos educacionais. Dessa forma, o grande desafio é: como definir maestria?

Utilizando a abordagem restrita, a maestria é atingida sempre que uma interação positiva é feita, ou seja, quando uma pessoa responde corretamente um problema qualquer sobre o assunto que está aprendendo. Essa maestria adquirida potencializa a sua capacidade cognitiva para responder corretamente novos desafios, mesmo que não sejam no mesmo nível de dificuldade daquele executado com sucesso anteriormente.

Adaptando para o caso deste repositório, temos disponíveis uma série de registros de problemas resolvidos por diversas pessoas em uma plataforma de ensino, contando com os campos de acerto, nível de dificuldade, área de conhecimento e data de realização. Montando uma série temporal para cada pessoa, podemos contar a quantidade de acertos acumulado em cada momento, assim como contar a quantidade de acertos em determinado nível de dificuldade (1 a 5). Dessa forma teremos um conjunto de dados que, para cada pessoa, em cada tentativa, apresenta o histórico prévio da quantidade de questões acertadas para aquela determinada área de conhecimento. Essa quantidade é a base para definir a maestria do respondente no momento em que resolve a atividade.

A partir da quantidade de acertos acumulados, atribui-se uma determinada pontuação para cada item (acerto geral e acerto em determinada dificuldade), de modo que a soma dessa pontuação resulta em um número que vamos chamar de maestria. Resumidamente, uma vez que a base passa a ser formatada em uma série temporal com acertos acumulados, é possível calcular a maestria em cada momento. Os valores da pontuação serão trabalhados no próximo notebook (param_tuning).

Separar a quantidade de acertos totais da quantidade de acertos em cada nível de dificuldade é uma estratégia para tentar reduzir ao máximo o problema de data leakage, uma vez que a maestria é uma função que depende do tempo e existem campos nulos relativos à dificuldade na base de dados original. A ideia é sempre pontuar quando um acerto for feito, independente do nível de dificuldade, porém incluir um pequeno bônus na pontuação de acordo com o nível de dificuldade. Isso faz com que, no cálculo de maestria, os valores nulos sejam ignorados nas colunas que contas a quantidade de questões em determinada dificuldade, porém o acerto ainda é computado no acumulado de acertos gerais, que independe da dificuldade.

In [7]:
data = []
for user in df["novo_user_id"].unique():
    # Para cada usuário, faz a ordenação das respostas, da mais antiga para
    # a mais recente para criar a série histórica adequada
    subset = df.query(f"novo_user_id == {user}").sort_values("created_at")
    areas = {}

    for i in range(0, len(subset)):
        # Carrega principais informações de processamento
        area = subset["knowledge_area_id"].values[i]
        difficulty = subset["difficulty"].values[i]

        # Instancia o dicionário de contagem de acertos
        if area not in areas.keys():
            areas[area] = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
        
        # Caso exista um acerto, computa +1 na contagem geral e, caso a
        # dificuldade esteja presente, computa +1 da dificuldade específica
        if subset["acertou"].values[i] == 1:
            areas[area][0] = areas[area][0] + 1
            if not np.isnan(difficulty):
                areas[area][difficulty] = areas[area][difficulty] + 1

        # Armazena na lista que criará o conjunto de dados final
        data.append({
            "user_id": subset["novo_user_id"].values[i],
            "area": subset["knowledge_area_id"].values[i],
            "difficulty": difficulty,
            "points": areas[area][0],
            "points1": areas[area][1],
            "points2": areas[area][2],
            "points3": areas[area][3],
            "points4": areas[area][4],
            "points5": areas[area][5],
            "right": subset["acertou"].values[i]
        })

# Armazena o conjunto processado no diretório do projeto, para utilização posterior
data = pd.DataFrame(data)
data.to_csv(f"{base_path}/data/mastery.csv", index=False)

## Referências

YU, L. Q; WILSON, R. C. & NASSAR, M. R. Adaptive learning is structure learning in time. Neuroscience & Biobehavioral Reviews, v. 128, p. 270-281, 2021.

CAVANAGH, T. et al. Constructing a Design Framework and Pedagogical Approach for Adaptive Learning in Higher Education: A Practitioner's Perspective. International Review of Research in Open and Distributed Learning, v. 21, n. 1, p. 173-197, 2020.

ARSOVIC, B. & STEFANOVIC, N. E-learning based on the adaptive learning model: case study in Serbia. Sādhanā, v. 45, 2020